In [1]:
# Import libraries, features and settings (not all of these are needed so pull what you need)

import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import io
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import preprocessing
plt.rc("font", size = 14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
sns.set(style="white")
sns.set(style="whitegrid", color_codes = True)

C:\Users\pasha\AppData\Local\Temp\ipykernel_24224\1367088338.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# read in the csvs as dataframes

df_train = pd.read_csv('../Data/train.csv')
df_test = pd.read_csv('../Data/test.csv')

df_train

In [ ]:
# get number of instances and features

num_instances, num_features = df_train.shape
print(f"Instances: {num_instances}")
print(f"Features: {num_features}")

In [ ]:
num_features = df_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = df_train.select_dtypes(include=['object']).columns
print("Numerical features:")
print(num_features)
print("Categorical features:")
print(cat_features)

In [ ]:
# Find features with missing values and the proportion of missing values

missing_values = df_train.isna().sum()
missing_prop = missing_values / len(df_train)
missing_data = pd.concat([missing_values, missing_prop], axis=1, keys=['Missing Values', 'Proportion'])
print(missing_data[missing_data['Missing Values'] > 0])

In [ ]:
num_features = df_train.select_dtypes(include=['int64', 'float64']).columns
for feature in num_features:
    num_unique = df_train[feature].nunique()
    if num_unique < 10:
        print(f"{feature} is a discrete feature with {num_unique} unique values")
    else:
        print(f"{feature} is a continuous feature with {num_unique} unique values")